In [39]:
#Библиотеки
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from scipy.stats import zscore

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam



from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import kagglehub


In [26]:


# Download latest version
path = kagglehub.dataset_download("goyaladi/twitter-bot-detection-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\HONOR\.cache\kagglehub\datasets\goyaladi\twitter-bot-detection-dataset\versions\2


In [27]:
data_path = path + "/bot_detection_data.csv"

# Чтение файла
df = pd.read_csv(data_path)

In [28]:
# Просмотр данных
print("Первые строки датасета:")
print(df.head())
print("\nИнформация о датасете:")
print(df.info())

Первые строки датасета:
   User ID        Username                                              Tweet  \
0   132131           flong  Station activity person against natural majori...   
1   289683  hinesstephanie  Authority research natural life material staff...   
2   779715      roberttran  Manage whose quickly especially foot none to g...   
3   696168          pmason  Just cover eight opportunity strong policy which.   
4   704441          noah87                      Animal sign six data good or.   

   Retweet Count  Mention Count  Follower Count  Verified  Bot Label  \
0             85              1            2353     False          1   
1             55              5            9617      True          0   
2              6              2            4363      True          0   
3             54              5            2242      True          1   
4             26              3            8438     False          1   

       Location           Created At            Hashtags

In [29]:
# Проверка на пропущенные значения
print(df.isnull().sum())

# Пример обработки:
# - Удаление строк с большим количеством пропусков
df.dropna(thresh=len(df.columns) - 2, inplace=True)

# - Заполнение пропусков
df['Location'] = df['Location'].fillna('Unknown')
df['Hashtags'] = df['Hashtags'].fillna('')
df['Retweet Count'] = df['Retweet Count'].fillna(0)


User ID              0
Username             0
Tweet                0
Retweet Count        0
Mention Count        0
Follower Count       0
Verified             0
Bot Label            0
Location             0
Created At           0
Hashtags          8341
dtype: int64


In [30]:
# Преобразование столбца 'Created At' в формат datetime
df['Created At'] = pd.to_datetime(df['Created At'])

# Преобразование логических значений
df['Verified'] = df['Verified'].astype(bool)

# Убедимся, что числовые данные имеют правильные типы
numeric_columns = ['Retweet Count', 'Mention Count', 'Follower Count', 'Bot Label']
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')


In [31]:
# Количество слов и длина твита
df['Tweet Word Count'] = df['Tweet'].apply(lambda x: len(str(x).split()))
df['Tweet Length'] = df['Tweet'].apply(lambda x: len(str(x)))

# Количество хэштегов
df['Hashtag Count'] = df['Hashtags'].apply(lambda x: len(str(x).split(',')))

# "Возраст" аккаунта (в днях)
import datetime
today = datetime.datetime.now()
df['Account Age (Days)'] = df['Created At'].apply(lambda x: (today - x).days)


In [32]:
# Удаление ненужных колонок
df.drop(columns=['User ID', 'Username', 'Location', 'Created At'], inplace=True)


In [33]:

df = pd.get_dummies(df, columns=['Verified'], drop_first=True)


In [34]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_columns = ['Retweet Count', 'Mention Count', 'Follower Count', 'Tweet Word Count', 
                  'Tweet Length', 'Hashtag Count', 'Account Age (Days)']
df[scaled_columns] = scaler.fit_transform(df[scaled_columns])


In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Преобразуем тексты из столбца Tweet в числовые векторы
tfidf = TfidfVectorizer(max_features=1000)  # Оставляем топ-1000 слов
tweet_features = tfidf.fit_transform(df['Tweet']).toarray()

# Преобразуем tweet_features в DataFrame и добавляем в основной DataFrame
tweet_features_df = pd.DataFrame(tweet_features, columns=[f'Tweet_Feature_{i}' for i in range(tweet_features.shape[1])])
df = pd.concat([df.reset_index(drop=True), tweet_features_df], axis=1)

# Удаляем оригинальный столбец Tweet
df = df.drop(columns=['Tweet'])


In [36]:
# Подсчитаем количество уникальных хэштегов
df['Unique Hashtag Count'] = df['Hashtags'].apply(lambda x: len(set(x.split(','))))

# Удаляем оригинальный столбец Hashtags
df = df.drop(columns=['Hashtags'])


In [37]:
# Проверка итогового DataFrame
print(df.info())
print(df.describe())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Columns: 979 entries, Retweet Count to Unique Hashtag Count
dtypes: bool(1), float64(976), int64(2)
memory usage: 373.1 MB
None
       Retweet Count  Mention Count  Follower Count     Bot Label  \
count   50000.000000   50000.000000    50000.000000  50000.000000   
mean        0.500056       0.502752        0.498860      0.500360   
std         0.291812       0.341713        0.287874      0.500005   
min         0.000000       0.000000        0.000000      0.000000   
25%         0.250000       0.200000        0.248775      0.000000   
50%         0.500000       0.600000        0.499150      1.000000   
75%         0.750000       0.800000        0.747100      1.000000   
max         1.000000       1.000000        1.000000      1.000000   

       Tweet Word Count  Tweet Length  Hashtag Count  Account Age (Days)  \
count      50000.000000  50000.000000        50000.0        50000.000000   
mean           0.44640

In [14]:
from sklearn.model_selection import train_test_split

# Отделяем признаки (X) и целевую переменную (y)
X = df.drop(columns=['Bot Label'])  # Все столбцы, кроме целевого
y = df['Bot Label']                # Целевая переменная

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Проверяем размеры наборов данных
print(f"Размер обучающей выборки: {X_train.shape[0]} строк")
print(f"Размер тестовой выборки: {X_test.shape[0]} строк")


Размер обучающей выборки: 40000 строк
Размер тестовой выборки: 10000 строк


In [15]:
# Модель 1: Логистическая регрессия
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)

LogisticRegression()

In [16]:
# Модель 2: Случайный лес
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [17]:
#Модель 3: нейронная сеть
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
# Определение модели
nn_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),  # Увеличено количество нейронов
    Dropout(0.4),  # Увеличен Dropout для регуляризации
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Выходной слой для бинарной классификации
])

# Компиляция модели
nn_model.compile(
    optimizer=Adam(learning_rate=0.001),  # Оптимизатор Adam с начальной скоростью обучения
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Колбэки: ранняя остановка и снижение скорости обучения
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)

# Обучение модели
history = nn_model.fit(
    X_train, y_train,
    epochs=50,  # Больше эпох, чтобы захватить сложные зависимости
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping, reduce_lr]
)

c:\Users\HONOR\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - accuracy: 0.4937 - loss: 0.6952 - val_accuracy: 0.4996 - val_loss: 0.6933 - learning_rate: 0.0010
Epoch 2/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.5144 - loss: 0.6927 - val_accuracy: 0.4941 - val_loss: 0.6939 - learning_rate: 0.0010
Epoch 3/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.5356 - loss: 0.6888 - val_accuracy: 0.4970 - val_loss: 0.6963 - learning_rate: 0.0010
Epoch 4/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.5600 - loss: 0.6812 - val_accuracy: 0.4959 - val_loss: 0.6996 - learning_rate: 0.0010
Epoch 5/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.6166 - loss: 0.6560 - val_accuracy: 0.5004 - val_loss: 0.7230 - learning_rate: 5.0000e-04
Epoch 6/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.6600 - loss: 0.6195 - val_accuracy: 0.4966 - val_loss: 0.7555 - learning_rate: 5.0000e-04


In [18]:
# Предсказания для классических моделей
lr_preds = lr_model.predict(X_test)
rf_preds = rf_model.predict(X_test)
nn_preds = (nn_model.predict(X_test) > 0.5).astype(int)

# Метрики для Логистической регрессии
print("\nЛогистическая регрессия:")
print("Accuracy:", accuracy_score(y_test, lr_preds))
print("F1-Score:", f1_score(y_test, lr_preds))
print("ROC-AUC:", roc_auc_score(y_test, lr_preds))

# Метрики для Random Forest
print("\nСлучайный лес:")
print("Accuracy:", accuracy_score(y_test, rf_preds))
print("F1-Score:", f1_score(y_test, rf_preds))
print("ROC-AUC:", roc_auc_score(y_test, rf_preds))

# Метрики для Нейросети
print("\nНейронная сеть:")
print("Accuracy:", accuracy_score(y_test, nn_preds))
print("F1-Score:", f1_score(y_test, nn_preds))
print("ROC-AUC:", roc_auc_score(y_test, nn_preds))

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step

Логистическая регрессия:
Accuracy: 0.5035
F1-Score: 0.5026545126715416
ROC-AUC: 0.5035016822410766

Случайный лес:
Accuracy: 0.4949
F1-Score: 0.48157651647336547
ROC-AUC: 0.49492087674936114

Нейронная сеть:
Accuracy: 0.5002
F1-Score: 0.6548819223864107
ROC-AUC: 0.49984175989872637
